In [1]:
import os
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from pathlib import Path

# Ensure NLTK stopwords are available
nltk.download('stopwords')

# Load stopwords globally for efficiency
stop_words = set(stopwords.words('english'))

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/21/25 18:00:10] INFO     Found credentials from IAM Role:                                   ]8;id=798774;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=101403;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# -------------------------------
# 1. Data Loading and Preprocessing
# -------------------------------

# Define dataset path (modify if needed)
data_path = Path("Restaurant_Reviews.csv")

# Load dataset with error handling
try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Dataset not found at {data_path}. Please check the path.")

# Drop unwanted columns (Modify if necessary)
unwanted_columns = ["7514"]  # Ensure this is a real column
df.drop(columns=[col for col in unwanted_columns if col in df.columns], inplace=True)

# Remove duplicates and null values
df.drop_duplicates(inplace=True)
df.dropna(subset=["Review", "Rating"], inplace=True)

# Normalize Ratings (Replace "Like" with 5, and handle other potential issues)
df['Rating'] = df['Rating'].replace({"Like": "5"}).astype(str)
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df.dropna(subset=["Rating"], inplace=True)

# Create sentiment labels
df['label'] = np.where(df['Rating'] <= 3.0, 'Negative', 'Positive')

# Define function to clean text
def clean_text(text):
    """Cleans review text by removing non-alphabet characters and stopwords."""
    text = re.sub(r"[^a-zA-Z']", ' ', str(text)).lower().split()
    return ' '.join([word for word in text if word not in stop_words])

# Apply text cleaning
df['clean_text'] = df['Review'].astype(str).apply(clean_text)

# Display cleaned data sample
print(df.head())

# Split data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=123)

        Restaurant              Reviewer  \
0  Beyond Flavours     Rusha Chakraborty   
1  Beyond Flavours  Anusha Tirumalaneedi   
2  Beyond Flavours       Ashok Shekhawat   
3  Beyond Flavours        Swapnil Sarkar   
4  Beyond Flavours                Dileep   

                                              Review  Rating  \
0  The ambience was good, food was quite good . h...     5.0   
1  Ambience is too good for a pleasant evening. S...     5.0   
2  A must try.. great food great ambience. Thnx f...     5.0   
3  Soumen das and Arun was a great guy. Only beca...     5.0   
4  Food is good.we ordered Kodi drumsticks and ba...     5.0   

                  Metadata             Time  Pictures     label  \
0   1 Review , 2 Followers  5/25/2019 15:54         0  Positive   
1  3 Reviews , 2 Followers  5/25/2019 14:20         0  Positive   
2  2 Reviews , 3 Followers  5/24/2019 22:54         0  Positive   
3    1 Review , 1 Follower  5/24/2019 22:11         0  Positive   
4  3 Reviews , 

In [3]:
# -------------------------------
# 2. Convert Data to AWS BlazingText Format
# -------------------------------

def create_bt_file(data, filename):
    """Convert data to AWS BlazingText format and save to file."""
    with open(filename, 'w') as f:
        for _, row in data.iterrows():
            f.write(f"__label__{row['label'].lower()} {row['clean_text']}\n")

# Save training and test data
train_filename, test_filename = 'train.txt', 'test.txt'
create_bt_file(train_data, train_filename)
create_bt_file(test_data, test_filename)

In [7]:
# -------------------------------
# 3. AWS BlazingText Training with SageMaker
# -------------------------------

# Set up SageMaker session and role
session = sagemaker.Session()

try:
    role = sagemaker.get_execution_role()
except Exception:
    import sagemaker.session
    role = sagemaker.session.Session().get_caller_identity_arn()

bucket = session.default_bucket()  # Use default S3 bucket

# Upload data to S3
prefix = 'blazingtext-sentiment'
train_input_path = session.upload_data(train_filename, bucket=bucket, key_prefix=f"{prefix}/train")
test_input_path = session.upload_data(test_filename, bucket=bucket, key_prefix=f"{prefix}/test")

# Get BlazingText container image
region = boto3.Session().region_name
bt_container = get_image_uri(region, 'blazingtext')

# Set up BlazingText estimator
bt = sagemaker.estimator.Estimator(
    image_uri=bt_container,
    role=role,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=session
)

# Configure hyperparameters
bt.set_hyperparameters(
    mode='supervised',
    epochs=10,
    learning_rate=0.05,
    vector_dim=100,
    min_count=2,
    word_ngrams=2
)

# Define data channels
train_channel = sagemaker.inputs.TrainingInput(train_input_path, content_type='text/plain')
test_channel = sagemaker.inputs.TrainingInput(test_input_path, content_type='text/plain')

# Train model
bt.fit({'train': train_channel, 'test': test_channel})

[02/21/25 18:06:10] WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=644004;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=647334;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Same images used for training and inference. Defaulting to image     ]8;id=421032;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=535689;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#391\391]8;;\
                             scope: inference.                                                                     

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=701372;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=418041;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=931259;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=878302;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: blazingtext-2025-02-21-18-06-10-897   ]8;id=832332;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=741804;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-02-21 18:06:13 Starting - Starting the training job...
..25-02-21 18:06:28 Starting - Preparing the instances for training.
..25-02-21 18:06:58 Downloading - Downloading input data.
2025-02-21 18:07:29 Training - Training image download completed. Training in progress.Arguments: train
[02/21/2025 18:07:37 WARNING 140269303150400] Loggers have already been setup.
[02/21/2025 18:07:37 WARNING 140269303150400] Loggers have already been setup.
/opt/amazon/python3.8/lib/python3.8/subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
[02/21/2025 18:07:37 INFO 140269303150400] nvidia-smi took: 0.025302648544311523 secs to identify 0 gpus
[02/21/2025 18:07:37 INFO 140269303150400] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[02/21/2025 18:07:37 INFO 140269303150400] Processing /opt/ml/input/d

In [10]:
# -------------------------------
# 4. Model Deployment and Prediction
# -------------------------------

# Deploy trained model
predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t3.medium')

def predict_sentiment(text):
    """Predict sentiment using deployed SageMaker model."""
    response = predictor.predict([text])
    return response

# Test sample reviews
print(predict_sentiment("The food was great and the service was excellent."))
print(predict_sentiment("I did not enjoy the meal; it was disappointing."))

[02/21/25 18:11:12] INFO     Creating model with name: blazingtext-2025-02-21-18-11-12-466          ]8;id=187045;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=476145;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 # -------------------------------                                                           │
│    4                                                                                             │
│    5 # Deploy trained model                                                                      │
│ ❱  6 predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t3.medium')               │
│    7                                                                                             │
│    8 def predict_sentiment(text):                                                                │
│    9 │   """Predict sentiment using deployed SageMaker model."""                                 │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/estimator.py:1722   │
│ in deploy                                                                                        │
│                                                                                                  │
│   1719 │   │   │   inference_tags=format_tags(tags), training_tags=self.tags                     │
│   1720 │   │   )                                                                                 │
│   1721 │   │                                                                                     │
│ ❱ 1722 │   │   return model.deploy(                                                              │
│   1723 │   │   │   instance_type=instance_type,                                                  │
│   1724 │   │   │   initial_instance_count=initial_instance_count,                                │
│   1725 │   │   │   serializer=serializer,                                                        │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1725 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1722 │   │   │   return None                                                                   │
│   1723 │   │                                                                                     │
│   1724 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1725 │   │   │   self._create_sagemaker_model(                                                 │
│   1726 │   │   │   │   instance_type=instance_type,                                              │
│   1727 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1728 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:985 in     │
│ _create_sagemaker_model                                                                          │
│                                                                                                  │
│    982 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    983 │   │   │   │   tags=format_tags(tags),                                                   │
│    984 │   │   │   )                                                                             │
│ ❱  985 │   │   │   self.sagemaker_session.create_model(**cr